In [1]:
import scrapy
import logging
import pandas as pd
from datetime import date
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

In [2]:
class PatientSpider(scrapy.Spider):
    name = "patient_check"
    start_urls = ['https://ncov.moh.gov.vn/web/guest/trang-chu']
    rows = []
    out = 'MOH_PaitentTable_' + date.today().strftime("%d_%m") + '.csv'

    def __init__(self):
        logging.getLogger('scrapy').propagate = False

    def close(self, reason):
        df_out = pd.DataFrame(self.rows)
        df_out.to_csv(self.out,index=False)
        print("Hoàn tất. Dữ liệu đã được xuất ra file " + self.out)

    def parse(self, response):
        for patient in response.xpath('//*[@class="table table-striped table-covid19"]//tbody/tr'):
            p_dict={'BN': patient.xpath('td[1]//text()').extract_first(),
                    'Age': patient.xpath('td[2]//text()').extract_first(),
                    'Location': patient.xpath('td[3]//text()').extract_first(),
                    'Status': patient.xpath('td[4]//text()').extract_first(),
                    'Nationality': patient.xpath('td[5]//text()').extract_first()}
            self.rows += [p_dict]

In [3]:
process = CrawlerProcess(get_project_settings())
process.crawl(PatientSpider)
process.start()

2021-05-23 17:55:32 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-05-23 17:55:32 [scrapy.utils.log] INFO: Versions: lxml 4.5.1.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.6.8 (tags/v3.6.8:3c6b436a57, Dec 24 2018, 00:16:47) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Windows-10-10.0.19041-SP0
2021-05-23 17:55:32 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-05-23 17:55:32 [scrapy.crawler] INFO: Overridden settings:
{}
2021-05-23 17:55:32 [scrapy.extensions.telnet] INFO: Telnet Password: eb4eb34f9bdf46f7
2021-05-23 17:55:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
Đang trích xuất dữ liệu, xin đợi trong giây lát: 100%|██████████| 5141/5141 [00:01<00:00, 4918.78it/s]
